In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

from sklearn.decomposition import FastICA, PCA

You need 3 microphones to decompose into 3 sources so that the ICA can create a system of equations the resolve to a unique solutions.

Given X a n x m matrix of mixed data (n = number of microphones, m = number of samples), then the ICA creates W a n x n matrix that produces S, a n x m matrix of n individual sources, with m samples. 

So if you have 3 sources, 3 microphones and record data for 2000 timesteps, n = 3, and m = 2000. 

Then:

S = W * X 

X needs to be a 3 x 2000 matrix, to be multipled by the 3 x 3 W matrix, to get the three sources in S. 

You can see this operation being done in the scikit learn source code for FastICA: https://github.com/scikit-learn/scikit-learn/blob/0fb307bf3/sklearn/decomposition/_fastica.py#L520-L524 

If you have less than three mixed sources, then the W matrix has to have more rows than columns, and the system of equations is overdetermined.

If you have more than three mixed sources, then the W matrix has to have less rows than columns, and the system of equations is undetermined. 

That's my best guess at any rate. As far as I can tell the FastICA code seems to support this interpretation. 

In [3]:
# https://scikit-learn.org/stable/auto_examples/decomposition/plot_ica_blind_source_separation.html#sphx-glr-auto-examples-decomposition-plot-ica-blind-source-separation-py

# Generate sample data
np.random.seed(0)
n_samples = 2000
time = np.linspace(0, 8, n_samples)

s1 = np.sin(2 * time)  # Signal 1 : sinusoidal signal
s2 = np.sign(np.sin(3 * time))  # Signal 2 : square signal
s3 = signal.sawtooth(2 * np.pi * time)  # Signal 3: saw tooth signal

S = np.c_[s1, s2, s3]
S += 0.2 * np.random.normal(size=S.shape)  # Add noise

S /= S.std(axis=0)  # Standardize data
# Mix data
A = np.array([[1, 1, 1], [0.5, 2, 1.0], [1.5, 1.0, 2.0]])  # Mixing matrix
X = np.dot(S, A.T)  # Generate observations

In [7]:
X.shape

(2000, 3)

In [8]:
# Compute ICA
ica = FastICA(n_components=3)
S_ = ica.fit_transform(X)  # Reconstruct signals
A_ = ica.mixing_  # Get estimated mixing matrix

# We can `prove` that the ICA model applies by reverting the unmixing.
assert np.allclose(X, np.dot(S_, A_.T) + ica.mean_)

# For comparison, compute PCA
pca = PCA(n_components=3)
H = pca.fit_transform(X)  # Reconstruct signals based on orthogonal components

In [10]:
print(S_.shape)

(2000, 3)


In [ ]:
 Plot results

plt.figure()

models = [X, S, S_, H]
names = ['Observations (mixed signal)',
         'True Sources',
         'ICA recovered signals',
         'PCA recovered signals']
colors = ['red', 'steelblue', 'orange']

for ii, (model, name) in enumerate(zip(models, names), 1):
    plt.subplot(4, 1, ii)
    plt.title(name)
    for sig, color in zip(model.T, colors):
        plt.plot(sig, color=color)

plt.tight_layout()
plt.show()